# Scraping

- code to scrap halodoc articles from api (json) and loop each page


In [266]:

import requests
from bs4 import BeautifulSoup, Tag
import pandas as pd
import re

base_url = 'https://magneto.api.halodoc.com/api/cms/articles?page_no='
results_url = []
results_title = []
results_content = []

# try 1 - 1000 => takes ~1 hour to run [DONE]
# halaman [api 1001 ke atas gada]

# loop each page
# for page in range(1001, 2000):
for page in range(1, 1000):
    # get url for each page and make request
    url = base_url + str(page)
    response = requests.get(url)
    data = response.json()

    # Loop through articles
    for article in data['result']:

        # Get URL for article
        article_url = article['source_url']

        # Make request to article URL
        article_response = requests.get(article_url)

        # parsing data
        soup = BeautifulSoup(article_response.text, features="html.parser")

        # get element by class
        container = soup.find(attrs={'class': 'article-page__article-body-wrapper'})
        if(container is None):
            print(f'{article_url} dilewati (variable container kosong) - page {page}')
            continue

        # get titles
        titles = container.findAll(attrs={'class': 'section-header__content-text-title'})

        # temp
        temp = soup.find(attrs={'class': "article__content"}).children
        
        # get contents
        contents = []
        for t in temp:
            if isinstance(t, Tag):
                # if t.name == 'p' and t.find('strong', text='Referensi'):
                #     break
                #     print(t.find('strong', text='Halodoc').get_text(strip=True))
                #     continue
                contents.append(t.get_text(strip=True))
            
        contents = ' '.join(contents)
        
        prefixes = ["Halodoc, Jakarta – ", "Halodoc, Jakarta –", "Halodoc, Jakarta", "Halodoc,Jakarta –", "Halodoc,Jakarta–", "Halodoc, Jakarta -", "Halodoc, Jakarta-"]
        for prefix in prefixes:
            if contents.startswith(prefix):
                contents = contents[len(prefix):].lstrip()
                break

        if "Referensi" in contents:
            contents = contents.split("Referensi")[0].rstrip()

        # remove some words
        contents = re.sub(r'\b(YukdownloadHalodocsekarang|Yuk,downloadaplikasiHalodocsekarang|AyodownloadHalodocsekarang)\b', '', contents)


        # append results
        results_url.append(article_url)
        for title in titles:
            results_title.append(title.text)
        results_content.append(contents)


https://www.halodoc.com/ketahui-5-jenis-masker-dan-fungsinya-untuk-wajah dilewati (variable container kosong) - page 65
https://www.halodoc.com/sering-diabaikan-ini-5-hal-yang-sering-memicu-depresi-pada-remaja dilewati (variable container kosong) - page 305
https://www.halodoc.com/harus-dihindari-agar-cepat-sembuh-ini-9-pantangan-sakit-tipes dilewati (variable container kosong) - page 352
https://www.halodoc.com/begini-cara-mendiagnosis-spina-bifida-pada-bayi dilewati (variable container kosong) - page 429
https://www.halodoc.com/ibu-ini-7-cara-ampuh-mencegah-infeksi-virus-pada-anak-anak dilewati (variable container kosong) - page 441
https://www.halodoc.com/perhatikan-ini-beda-gejala-psikosis-dan-skizofrenia dilewati (variable container kosong) - page 529
https://www.halodoc.com/inilah-kiat-mudah-memberikan-asi-saat-sudah-masuk-kerja dilewati (variable container kosong) - page 734
https://www.halodoc.com/cara-mencegah-dan-mengatasi-bau-mulut-saat-berpuasa dilewati (variable container 

In [267]:
# create dataframe
df = pd.DataFrame({
                   'url': results_url, 
                   'title': results_title, 
                   'title_count' : [len(title.split()) for title in results_title],
                   'content': results_content, 
                   'content_count' : [len(content.split()) for content in results_content]
                   })

df.head()

,url,title,title_count,content,content_count
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"Ibu, Perhatikan 5 Hal Ini saat Memilih Car Se...",10,"Bagi orang tua yang memiliki kendaraan, memili...",540
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"Catat, Ini 6 Tips Menyimpan Beras agar Tidak ...",9,Sebagai makanan pokok beras harus disimpan den...,564
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"Waspada, Ini 6 Bahaya Asap Rokok bagi Kesehat...",9,Merokok tidak hanya berbahaya untuk dirimu sen...,649
3,https://www.halodoc.com/sering-terkena-polusi-...,"Sering Terkena Polusi, Ketahui Berbagi Cara B...",8,"Sehari-hari, manusia terpapar polusi dalam jum...",523
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,Ini Risiko dan Manfaat Implan Payudara yang P...,9,Implan payudara adalah prosedur bedah kosmetik...,547


In [268]:
# create dataframe
df = pd.DataFrame({
                   'url': results_url, 
                   'title': results_title, 
                   'title_count' : [len(title.split()) for title in results_title],
                   'content': results_content, 
                   'content_count' : [len(content.split()) for content in results_content]
                   })

# save to csv with good format
df.to_csv('halodocnew_1.csv', index=False, encoding='utf-8-sig')

In [269]:
# read halodoc.csv to dataframe

df = pd.read_csv('halodocnew_1.csv')
df

,url,title,title_count,content,content_count
0,https://www.halodoc.com/ibu-perhatikan-5-hal-i...,"Ibu, Perhatikan 5 Hal Ini saat Memilih Car Se...",10,"Bagi orang tua yang memiliki kendaraan, memili...",540
1,https://www.halodoc.com/catat-ini-6-tips-menyi...,"Catat, Ini 6 Tips Menyimpan Beras agar Tidak ...",9,Sebagai makanan pokok beras harus disimpan den...,564
2,https://www.halodoc.com/waspada-ini-6-bahaya-a...,"Waspada, Ini 6 Bahaya Asap Rokok bagi Kesehat...",9,Merokok tidak hanya berbahaya untuk dirimu sen...,649
3,https://www.halodoc.com/sering-terkena-polusi-...,"Sering Terkena Polusi, Ketahui Berbagi Cara B...",8,"Sehari-hari, manusia terpapar polusi dalam jum...",523
4,https://www.halodoc.com/ini-risiko-dan-manfaat...,Ini Risiko dan Manfaat Implan Payudara yang P...,9,Implan payudara adalah prosedur bedah kosmetik...,547
...,...,...,...,...,...
9974,https://www.halodoc.com/kata-dokter-benarkah-m...,Kata Dokter: Benarkah Minum Susu Bikin Kulit ...,9,“Minum susu tidak bisa membuat kulit bayi menj...,637
9975,https://www.halodoc.com/beginilah-cara-tepat-g...,Beginilah Cara Tepat Grooming Anak Anjing,6,Caragroominganjing mungkin tidak diketahui ole...,475
9976,https://www.halodoc.com/apa-saja-manfaat-kelap...,Apa Saja Manfaat Kelapa Hijau untuk Ibu Hamil...,9,"“Selama hamil, ibu bisa mendapatkan manfaat ke...",550
9977,https://www.halodoc.com/5-cara-merawat-kesehat...,5 Cara Merawat Kesehatan Kulit Ibu setelah Us...,9,"“Memasuki usia 50 tahun, ibu akan menghadapi b...",640
